In [1]:
import pandas as pd
import requests

In [2]:
community = pd.read_csv('clean_data/house_data.csv')
community.head()

,小区名字,小区地址,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent
0,骏景花园,天河-棠下-中山大道190号,49967,2.17,1,2.80,40.0
1,美林海岸花园,天河-员村-棕林街61号,77082,2.44,1,1.20,33.4
2,盈彩美居,天河-东圃-盈溪路,54560,2.88,1,1.20,30.0
3,兰亭盛荟,天河-东圃-东圃明镜路,70093,3.80,1,3.50,62.0
4,中海康城花园,天河-黄村-广园快速路,45544,1.74,1,2.37,40.0


In [3]:
# Baidu地图API的URL和AK
url = "https://api.map.baidu.com/geocoding/v3"
ak = 'ExRXnYlmwF71mD6Huayc4GOPS0aNqzqY'

In [4]:
# 定义一个函数通过地址获取经纬度
def get_geocode(address, ak):
    params = {
        "address": address,
        "output": "json",
        "ak": ak
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == 0:
            lat = result['result']['location']['lat']
            lng = result['result']['location']['lng']
            return lat, lng
    return None, None

In [5]:
community['latitude'] = None
community['longitude'] = None

In [6]:
# 对每个小区地址调用API并存储结果
# .at 方法用于频繁的单值操作
for index in community.index:
    lat, lng = get_geocode(community.loc[index]['小区地址'], ak)
    community.at[index, 'latitude'] = lat
    community.at[index, 'longitude'] = lng

In [7]:
community.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   小区名字         3081 non-null   object 
 1   小区地址         3081 non-null   object 
 2   均价           3081 non-null   int64  
 3   容积率          3081 non-null   float64
 4   有无停车位        3081 non-null   int64  
 5   物业费_元/平方米/月  3081 non-null   float64
 6   绿化率_percent  3081 non-null   float64
 7   latitude     2843 non-null   object 
 8   longitude    2843 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 216.8+ KB


In [8]:
community_with_coords = community.dropna(axis=0)

In [9]:
community_with_coords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2843 entries, 0 to 3080
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   小区名字         2843 non-null   object 
 1   小区地址         2843 non-null   object 
 2   均价           2843 non-null   int64  
 3   容积率          2843 non-null   float64
 4   有无停车位        2843 non-null   int64  
 5   物业费_元/平方米/月  2843 non-null   float64
 6   绿化率_percent  2843 non-null   float64
 7   latitude     2843 non-null   object 
 8   longitude    2843 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 222.1+ KB


In [ ]:
# 读取广州市中心城区的shp文件 用于剔除不在中心城区的错误定位
center_city_data = gpd.read_file('guangzhou_center_city.shp')
# 根据经纬度坐标创建Point几何对象列
community_gdf = gpd.GeoDataFrame(communities, geometry=gpd.points_from_xy(communities.longitude, communities.latitude))
# 使用空间连接（spatial join）来筛选出在中心城区内的小区，同样用'within'表示在范围内
filtered_residential_gdf = gpd.sjoin(community_gdf, center_city_data, how='inner', op='within')

In [11]:
community_with_coords.to_csv('communities_with_coords.csv', index=False)